In [57]:
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import random
import os

In [58]:
df = pd.read_csv('train_set.csv')
df

,id_img,path,label
0,0,../data/images/train/angry\0.jpg,angry
1,1,../data/images/train/angry\1.jpg,angry
2,10,../data/images/train/angry\10.jpg,angry
3,10002,../data/images/train/angry\10002.jpg,angry
4,10016,../data/images/train/angry\10016.jpg,angry
...,...,...,...
28816,9969,../data/images/train/surprise\9969.jpg,surprise
28817,9985,../data/images/train/surprise\9985.jpg,surprise
28818,9990,../data/images/train/surprise\9990.jpg,surprise
28819,9992,../data/images/train/surprise\9992.jpg,surprise


In [59]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

df['target'] = label_encoder.fit_transform(df['label'])

In [60]:
df['path'] = df['path'].str.replace('\\', '/')
df['path'] = df['path'].str.replace('../data/images/train/', './data/images/train/')

def cargar_y_convertir_imagen(ruta_imagen):
    image = cv2.imread(ruta_imagen, 0)
    smallimage = cv2.resize(image, (48, 48))
    image_array = np.array(smallimage)
    return image_array

df['imagenes'] =df['path'].apply(cargar_y_convertir_imagen)

In [61]:
df['path'] = df['path'].str.replace('\\', '/')
df['path'] = df['path'].str.replace('../data/images/train/', './data/images/train/')
def cargar_y_convertir_imagen(ruta_imagen):
    image = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)  # Lee la imagen en escala de grises
    smallimage = cv2.resize(image, (48, 48))
    
    # Convierte la imagen a RGB replicando el canal de escala de grises
    image_rgb = cv2.cvtColor(smallimage, cv2.COLOR_GRAY2RGB)
    
    return image_rgb


df['imagenes'] = df['path'].apply(cargar_y_convertir_imagen)

In [62]:
from sklearn.utils import shuffle
df = shuffle(df, random_state=42)

In [63]:
df['imagenes'][1].shape

(48, 48, 3)

In [64]:
df['imagenes'] = df['imagenes'].apply(lambda x: np.array(x).reshape(48, 48, 3))

X = np.stack(df['imagenes'].to_numpy())
y = df['target']

X = X / 255.0  

In [65]:
from keras.callbacks import ModelCheckpoint
mcheckpoint = ModelCheckpoint("callback_model.h5")

In [66]:
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(patience=5)

In [47]:

modelo_cnn = Sequential()
modelo_cnn.add(Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 3)))
modelo_cnn.add(MaxPooling2D((2, 2)))
modelo_cnn.add(Conv2D(128, (3, 3), activation='relu'))
modelo_cnn.add(MaxPooling2D((2, 2)))
modelo_cnn.add(Flatten())
modelo_cnn.add(Dense(128, activation='relu'))
modelo_cnn.add(Dropout(0.5))
modelo_cnn.add(Dense(64, activation='relu'))
modelo_cnn.add(Dropout(0.5))
modelo_cnn.add(Dense(7, activation='sigmoid'))  

opt = Adam(learning_rate=0.0001)
modelo_cnn.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

modelo_cnn.summary()

modelo_cnn.fit(X, y, epochs=50, batch_size=48, validation_split=0.2, callbacks=[earlystop, mcheckpoint])

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 26, 26, 64)        1792      
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 13, 13, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 11, 11, 128)       73856     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 5, 5, 128)         0         
 ng2D)                                                           
                                                                 
 flatten_10 (Flatten)        (None, 3200)              0         
                                                                 
 dense_30 (Dense)            (None, 128)             

c:\Users\migue\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


824/824 [==============================] - 10s 12ms/step - loss: 1.8118 - accuracy: 0.2518 - val_loss: 1.7255 - val_accuracy: 0.2973
Epoch 3/50
824/824 [==============================] - 10s 12ms/step - loss: 1.7307 - accuracy: 0.3053 - val_loss: 1.6424 - val_accuracy: 0.3684
Epoch 4/50
824/824 [==============================] - 10s 12ms/step - loss: 1.6764 - accuracy: 0.3409 - val_loss: 1.6006 - val_accuracy: 0.3847
Epoch 5/50
824/824 [==============================] - 10s 12ms/step - loss: 1.6350 - accuracy: 0.3553 - val_loss: 1.5529 - val_accuracy: 0.4040
Epoch 6/50
824/824 [==============================] - 10s 12ms/step - loss: 1.6007 - accuracy: 0.3711 - val_loss: 1.5199 - val_accuracy: 0.4201
Epoch 7/50
824/824 [==============================] - 10s 12ms/step - loss: 1.5769 - accuracy: 0.3879 - val_loss: 1.5017 - val_accuracy: 0.4198
Epoch 8/50
824/824 [==============================] - 10s 12ms/step - loss: 1.5514 - accuracy: 0.3963 - val_loss: 1.4809 - val_accuracy: 0.4337
Epo

In [67]:
#prueba con 48x48x3
modelo_cnn = Sequential()
modelo_cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)))
modelo_cnn.add(MaxPooling2D((2, 2)))
modelo_cnn.add(Conv2D(64, (3, 3), activation='relu'))
modelo_cnn.add(MaxPooling2D((2, 2)))
modelo_cnn.add(Conv2D(128, (3, 3), activation='relu'))
modelo_cnn.add(MaxPooling2D((2, 2)))
modelo_cnn.add(Flatten())
modelo_cnn.add(Dense(256, activation='relu'))
modelo_cnn.add(Dropout(0.5))
modelo_cnn.add(Dense(128, activation='relu'))
modelo_cnn.add(Dropout(0.5))
modelo_cnn.add(Dense(7, activation='sigmoid'))  

opt = Adam(learning_rate=0.0001)
modelo_cnn.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

modelo_cnn.summary()

modelo_cnn.fit(X, y, epochs=50, batch_size=48, validation_split=0.2, callbacks=[earlystop, mcheckpoint])

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 46, 46, 32)        896       
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 23, 23, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_13 (Conv2D)          (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 10, 10, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_14 (Conv2D)          (None, 8, 8, 128)         73856     
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 4, 4, 128)       

c:\Users\migue\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


481/481 [==============================] - 13s 28ms/step - loss: 1.7756 - accuracy: 0.2780 - val_loss: 1.6870 - val_accuracy: 0.3384
Epoch 3/50
481/481 [==============================] - 13s 27ms/step - loss: 1.6632 - accuracy: 0.3472 - val_loss: 1.5800 - val_accuracy: 0.3853
Epoch 4/50
481/481 [==============================] - 13s 27ms/step - loss: 1.5955 - accuracy: 0.3773 - val_loss: 1.5094 - val_accuracy: 0.4191
Epoch 5/50
481/481 [==============================] - 13s 27ms/step - loss: 1.5455 - accuracy: 0.4019 - val_loss: 1.4768 - val_accuracy: 0.4364
Epoch 6/50
481/481 [==============================] - 13s 27ms/step - loss: 1.5037 - accuracy: 0.4172 - val_loss: 1.4353 - val_accuracy: 0.4598
Epoch 7/50
481/481 [==============================] - 13s 27ms/step - loss: 1.4598 - accuracy: 0.4383 - val_loss: 1.3951 - val_accuracy: 0.4715
Epoch 8/50
481/481 [==============================] - 13s 27ms/step - loss: 1.4284 - accuracy: 0.4539 - val_loss: 1.3679 - val_accuracy: 0.4777
Epo

In [68]:
from tensorflow.keras.models import save_model

modelo_cnn.save('modelo_cnn28x28.h5')

In [69]:
test=pd.read_csv('test_set.csv')

In [70]:
test['path'] = test['id_img'].apply(lambda x: f'./data/images/test/{x}.jpg')

In [71]:
test['imagenes'] = test['path'].apply(cargar_y_convertir_imagen)

In [72]:
from tensorflow.keras.models import load_model

# Cargar el modelo entrenado
modelo_cnn = load_model('modelo_cnn28x28.h5')  # Reemplaza 'modelo_cnn.h5' con el nombre de tu modelo guardado

# Aplanar las imágenes y remodelarlas a 48x48
test['imagenes'] = test['imagenes'].apply(lambda x: np.array(x).reshape(48, 48, 3))

# Normalizar las imágenes
test['imagenes'] = test['imagenes'] / 255.0

# Hacer predicciones en el conjunto de prueba
X_test = np.stack(test['imagenes'].to_numpy())
predicciones = modelo_cnn.predict(X_test)

# Obtener las etiquetas predichas
predicciones_labels = np.argmax(predicciones, axis=1)

# Agregar las etiquetas predichas al DataFrame test
test['label_encoded_pred'] = predicciones_labels

221/221 [==============================] - 1s 6ms/step


In [73]:
test.drop(columns=(['path', 'imagenes']), inplace=True)

In [74]:
test['label'] = label_encoder.inverse_transform(predicciones_labels)

In [75]:
submission = pd.DataFrame({'id_img': test['id_img'], 'label': test['label']})

submission.to_csv('submission.csv', index=False)